# Revisiting the mushroom dataset using simple_neural_net

Here, we look at modeling the mushroom dataset using a neural network that has been coded from scratch using simple python vectorization with Numpy.

The data pre-processing follows exactly what was done in the mushroom_classification notebook, and details can be found there.

This notebook is a simple test to see if we can achieve good results using simple_neural_net.

### Load libraries

In [3]:
import pandas as pd
import numpy as np
from data_science_projects import simple_neural_net as snn
from sklearn.model_selection import train_test_split

### Load data and pre-process

In [4]:
df = pd.read_csv('../../data/mushrooms.csv')

df_ring = df['ring-number']
df_ring.replace({'n':0, 'o':1, 't':2}, inplace=True)

df = df.drop(columns=['ring-number'])
df_onehot = pd.get_dummies(df, drop_first=True)
# I am creating new dataframes, since the data is not too big and it makes it easier to run with small changes later

df_onehot['ring-number'] = df_ring

cut_correlation = 0.3
# A cut of 0.3 was found to give the best scores for both cross-validation
# on the training set and the test set score for the decision tree.

corr = df_onehot.corr('pearson')
cut_df_onehot = df_onehot[corr['class_p'][corr['class_p'].abs()>cut_correlation].index]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(cut_df_onehot.drop('class_p', 1), cut_df_onehot['class_p'], test_size = 0.1, random_state=10)

### Simple neural network model

In [6]:
layer_dims = [X_train.T.shape[0], X_train.T.shape[0], 1]

init_scale = 2.0 / np.sqrt(layer_dims)

parameters = snn.L_layer_model(np.array(X_train.T),
                               np.array(y_train).reshape([1,len(y_train)]),
                               layer_dims,
                               seed=1,
                               init_scale=init_scale,
                               learning_rate=0.01,
                               rel_tol=0.001,
                               hidden_active="relu",
                               outer_active="sigmoid",
                               cost_func="entropy",
                               print_cost = 1000,
                               max_iter=10000,
                               batch_size=0)

Cost after iteration 0: 9.240752e-01
Cost after iteration 1000: 5.324717e-04
Cost after iteration 2000: 1.145344e-04
Cost has not changed for 10 iterations
Time for run = 0 mins : 12 secs


In [7]:
y_pred, caches = snn.L_model_forward(np.array(X_test.T), parameters, hidden_active='relu', outer_active='sigmoid')
y_pred = np.where(y_pred<0.5, 0, 1)

In [9]:
accuracy = len(np.where(y_pred[0]==np.array(y_test))[0])/len(y_test)
print('accuracy of neural network = ', accuracy)

accuracy of neural network =  1.0


We see that we have again acieved 100% prediction, this time using a very shallow neural network with only one hidden layer.